In [1]:
import math
from collections import defaultdict
import random

In [42]:
class Dataset:
    def __init__(self, images_file, labels_file, lines_per_image = 28):
        self.images = []
        self.labels = []
        self.counts_by_label = defaultdict(int)
        self.priors = {}
        with open(images_file) as file:
            EOF = False
            while not EOF:
                image = []
                for i in range(lines_per_image):
                    line = file.readline()
                    if not line:
                        EOF = True
                        break
                    image.append(list(line))
                if EOF:
                    break
                self.images.append(image)
        with open(labels_file) as file:
            for i,label in enumerate(file):
                label = int(label)
                self.labels.append(label)
                self.counts_by_label[label]+=1
    def display(self, i):
        print("".join(map(lambda x: "".join(x),self.images[i])))
    def __len__(self):
        return len(self.labels)
    
    def shuffleData(self):
        order = list(zip(self.labels, self.images))
        random.shuffle(order)
        tempL, tempI = zip(*order) #zip turns them into giant tuples, want in list form
        self.labels = list(tempL)
        self.images = list(tempI)
    
    
class Perceptron:
    def __init__(self, label):
        self.label = label                                
        self.bias = 0 #random.random() #commented because idk which one to ultimately use
        self.weightVector = [[0 for i in range(28)] for j in range(28)]
        self.totalCount = 0
        
        
    def display(self):
        print('\n'.join(str(self.weightVector[i]) for i in range(28)))
        
    def trainVectorOnCorrect(self, image):
        #self.totalCount += 1 
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                    self.weightVector[i][j] += 2 #not sure what exact values should be
    def trainVectorOnIncorrect(self, image):
        #self.totalCount -= 1
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                     self.weightVector[i][j] -= 2 
                        
    def imageEvaluation(self, image):
        likelihood = 0
        for i in range(28):
            for j in range(28):
                if(image[i][j] == ' '):
                    likelihood -= self.weightVector[i][j]
                elif image[i][j] != '\n':
                    likelihood += self.weightVector[i][j]
        return likelihood

In [43]:
def retrainLoop(image, actualLabel):
    iterationLimit = 10
    while(True):
        if(iterationLimit < 0):
            break
        chances = [perceptrons[j].imageEvaluation(image) for j in range(10)]
        bestGuess = chances.index(max(chances))
        #print(chances)
        #print("is: " +  str(actualLabel) + " guessed: " + str(bestGuess))
        if bestGuess == actualLabel:
            perceptrons[actualLabel].trainVectorOnCorrect(image)
            break
        else :
            perceptrons[actualLabel].trainVectorOnIncorrect(perceptrons[bestGuess].weightVector)
            perceptrons[actualLabel].trainVectorOnCorrect(image)
        iterationLimit -= 1

In [44]:
trainingData = Dataset("trainingimages", "traininglabels")
perceptrons = [None]*10
for i in range(10):
    perceptrons[i] = Perceptron(i)

In [45]:
#first training pass, epoch -1 so to speak

for i in range(len(trainingData.images)):
    currDigit = trainingData.labels[i]    
    currPercept = perceptrons[currDigit]
    currPercept.trainVectorOnCorrect(trainingData.images[i])
print("done with epoch -1")  


#
# RUNNING ALL EPOCHS TAKES A VERY LONG TIME
#
epochs = 3
for q in range(epochs):
    correctGuesses = 0
    for i in range(len(trainingData.images)):
        currImage = trainingData.images[i]
        actualLabel = trainingData.labels[i]
        for j in range(10):
            activate = perceptrons[j].imageEvaluation(currImage)
            if activate > 1000: 
                if j == actualLabel :
                    correctGuesses +=1
                    perceptrons[actualLabel].trainVectorOnCorrect(currImage)
                else :
                    retrainLoop(currImage, actualLabel)
            elif j== actualLabel:
                perceptrons[actualLabel].trainVectorOnCorrect(currImage)

    print("epoch number: " + str(q) + ", accuracy: " + str(correctGuesses/len(trainingData.images)))

done with epoch -1
epoch number: 0, accuracy: 0.9942
epoch number: 1, accuracy: 1.0
epoch number: 2, accuracy: 1.0


In [47]:
testData = Dataset("testimages", "testlabels")
confusion_matrix_count = [[0 for i in range(10)] for j in range(10)]

indices = len(testData.images)
for i in range(indices):
    currImage = testData.images[i]
    chances = [perceptrons[j].imageEvaluation(currImage) for j in range(10)]
    bestGuess = chances.index(max(chances))
    actualLabel = testData.labels[i]
    if bestGuess == actualLabel :
        #perceptrons[actualLabel].trainVectorOnCorrect(currImage)
        confusion_matrix_count[bestGuess][bestGuess] +=1
    else :
        #retrainLoop(currImage, actualLabel)
        confusion_matrix_count[actualLabel][bestGuess] +=1


In [48]:
for i in range(10):
    print(confusion_matrix_count[i])
    
for i in range(10):
    correct = confusion_matrix_count[i][i]
    total = sum(confusion_matrix_count[i])
    rate = 100* correct/total
    print("Label: "+ str(i) + " was accurate at a rate of: " + str(rate) + "%" )

[60, 0, 2, 0, 0, 21, 5, 0, 2, 0]
[0, 103, 2, 0, 0, 2, 1, 0, 0, 0]
[1, 2, 81, 4, 1, 2, 7, 0, 3, 2]
[0, 0, 0, 74, 0, 18, 0, 3, 2, 3]
[0, 1, 2, 0, 92, 1, 3, 1, 1, 6]
[0, 1, 0, 2, 1, 84, 0, 1, 2, 1]
[0, 3, 4, 0, 5, 9, 68, 0, 2, 0]
[0, 4, 11, 1, 1, 5, 0, 72, 0, 12]
[0, 1, 2, 12, 0, 15, 0, 1, 66, 6]
[0, 2, 1, 4, 6, 5, 0, 2, 1, 79]
Label: 0 was accurate at a rate of: 66.66666666666667%
Label: 1 was accurate at a rate of: 95.37037037037037%
Label: 2 was accurate at a rate of: 78.64077669902913%
Label: 3 was accurate at a rate of: 74.0%
Label: 4 was accurate at a rate of: 85.98130841121495%
Label: 5 was accurate at a rate of: 91.30434782608695%
Label: 6 was accurate at a rate of: 74.72527472527473%
Label: 7 was accurate at a rate of: 67.9245283018868%
Label: 8 was accurate at a rate of: 64.07766990291262%
Label: 9 was accurate at a rate of: 79.0%
